In [ ]:
# @title INSTALAÇÃO DOS MÓDULOS NECESSÁRIOS

# Remver comentário da linha abaixo, caso seja necessário instalar o Pandas
#!pip install pandas -q

In [25]:
# @title IMPORTAÇÃO DAS BIBLIOTECAS

import sqlite3 as sq
import pandas as pd
import json

In [26]:
# @title CONEXÃO COM A BASE DE DADOS SQLITE

BASE = "datatran"
conn = sq.connect(f"{BASE}.db", check_same_thread=False)

In [31]:
# @title CONSULTA DOS NOMES DE ESTADOS

query = f"SELECT * FROM estados"
lista_estados = pd.read_sql(query, conn)['estado'].to_list()
lista_estados

['Acre',
 'Alagoas',
 'Amapá',
 'Amazonas',
 'Bahia',
 'Ceará',
 'Distrito Federal',
 'Espírito Santo',
 'Goiás',
 'Maranhão',
 'Mato Grosso',
 'Mato Grosso do Sul',
 'Minas Gerais',
 'Pará',
 'Paraíba',
 'Paraná',
 'Pernambuco',
 'Piauí',
 'Rio Grande do Norte',
 'Rio Grande do Sul',
 'Rio de Janeiro',
 'Rondônia',
 'Roraima',
 'Santa Catarina',
 'São Paulo',
 'Sergipe',
 'Tocantins']

In [64]:
# @title CONSULTA POR ESTADO: NÚMEROS DE ACIDENTES

query = f"""
    SELECT e.estado, COUNT(*)
    FROM estados AS e
        INNER JOIN ocorrencias AS o
            ON e.cod = o.cod_estado 
        INNER JOIN acidentes AS a 
            ON a.cod = o.cod_acidente
    GROUP BY e.estado
    ORDER BY COUNT(*) DESC;
    """
df = pd.read_sql_query(query, con=conn)
df.columns = ['Estado','Acidentes']

dict_acidentes = {}
dados = zip(df['Estado'], df['Acidentes'])
for estado, acidentes in dados:
    dict_acidentes[estado] = acidentes
    
dict_acidentes

{'Minas Gerais': 8290,
 'Santa Catarina': 7589,
 'Paraná': 7377,
 'Rio Grande do Sul': 4929,
 'Rio de Janeiro': 4704,
 'São Paulo': 4385,
 'Bahia': 3440,
 'Goiás': 3112,
 'Pernambuco': 2746,
 'Espírito Santo': 2278,
 'Mato Grosso': 2175,
 'Mato Grosso do Sul': 1638,
 'Paraíba': 1474,
 'Rondônia': 1378,
 'Ceará': 1373,
 'Rio Grande do Norte': 1287,
 'Piauí': 1114,
 'Maranhão': 1084,
 'Distrito Federal': 922,
 'Pará': 831,
 'Alagoas': 595,
 'Tocantins': 568,
 'Sergipe': 511,
 'Roraima': 229,
 'Acre': 224,
 'Amapá': 180,
 'Amazonas': 114}

In [68]:
# @title CONSULTA POR ESTADO: NÚMEROS DE MORTES

query = f"""
    SELECT e.estado, sum(a.mortos)
    FROM estados AS e
        INNER JOIN ocorrencias AS o
            ON e.cod = o.cod_estado 
        INNER JOIN acidentes AS a 
            ON a.cod = o.cod_acidente
    GROUP BY e.estado
    ORDER BY sum(a.mortos) DESC;
    """
df = pd.read_sql_query(query, con=conn)
df.columns = ['Estado','Fatalidades']

dict_fatalidades = {}
dados = zip(df['Estado'], df['Fatalidades'])
for estado, fatalidades in dados:
    dict_fatalidades[estado] = fatalidades

dict_fatalidades

{'Minas Gerais': 701,
 'Paraná': 571,
 'Bahia': 528,
 'Santa Catarina': 350,
 'Rio Grande do Sul': 334,
 'Pernambuco': 305,
 'Goiás': 291,
 'Mato Grosso': 263,
 'Rio de Janeiro': 257,
 'São Paulo': 223,
 'Maranhão': 211,
 'Pará': 183,
 'Mato Grosso do Sul': 167,
 'Espírito Santo': 154,
 'Ceará': 148,
 'Piauí': 137,
 'Paraíba': 129,
 'Rio Grande do Norte': 94,
 'Rondônia': 86,
 'Tocantins': 81,
 'Alagoas': 54,
 'Sergipe': 47,
 'Distrito Federal': 44,
 'Roraima': 24,
 'Amapá': 20,
 'Amazonas': 19,
 'Acre': 18}

In [100]:
# @title CONSULTA POR ESTADO: VIA MAIS LETAL
'''
dict_via = {}
dados = zip(df['Estado'], df['Rodovia'])
for estado, rodovia in dados:
    dict_via[estado] = rodovia

dict_via'''

query = f"""
SELECT estado, rodovia, s.SumMortos
FROM
    (SELECT e.estado, a.rodovia, sum(a.mortos) AS SumMortos
    FROM estados AS e
    INNER JOIN ocorrencias AS o
        ON e.cod = o.cod_estado
    INNER JOIN acidentes AS a 
        ON a.cod = o.cod_acidente
    GROUP BY e.estado, a.rodovia
    ORDER BY sum(a.mortos) DESC) AS s
WHERE
    s.SumMortos = (SELECT MAX(a.mortos)
                   FROM acidentes as a
                   WHERE a.rodovia = s.rodovia); 
"""
     
df = pd.read_sql_query(query, con=conn)
#df.columns = ['Estado','Rodovia', 'Mortos']
df


,estado,rodovia,SumMortos
0,Pernambuco,116,6
1,Piauí,20,6
2,Minas Gerais,364,5
3,Paraná,158,4
4,Paraná,280,4
5,Santa Catarina,158,4
6,São Paulo,459,4
7,Amazonas,174,3
8,Minas Gerais,356,3
9,Piauí,402,2


In [7]:
dados_json = []
for estado in lista_estados:
    
    acidentes = dict_acidentes[estado]
    fatalidades = dict_fatalidades[estado]
    via_mais_letal = dict_via[estado]
    principal_causa = dict_causa[estado]

    dado_estado = {'estado': estado,
    'acidentes': acidentes,
    'fatalidades': fatalidades,
    'via_mais_perigosa': via_mais_letal,
    'principal_causa': principal_causa}

    dados_json.append(dado_estado)


,Rodovia,Mortos
0,116,640
1,101,603
2,153,225
3,163,221
4,381,188
